分为两步：  
* zz2dis  
* dis2ks  

In [1]:
import sys
sys.path.append('/home/ian/code/github')

In [2]:
from utils.pymongo_wrapper import pymongo_wrapper

In [3]:
pmw = pymongo_wrapper()

In [4]:
zztable = pmw.get_collection('jiankang39','zznew')

In [6]:
zzlist = list(pmw.find_all(zztable,fieldlist=['症状名称','相关疾病']))
zzlist[:1]

[{'症状名称': '多梦',
  '相关疾病': ['睡眠障碍',
   '缺血缺氧性脑病',
   '脑外伤伴发的精神障碍',
   '癔症',
   '慢性乙肝',
   '慢性疲劳综合症',
   '体质性低血压',
   '女子梦交',
   '肝火上炎证',
   '肝阳上亢',
   '阴虚火旺',
   '阴虚发热',
   '心阴虚',
   '心血虚',
   '结肠炎',
   '黧黑斑',
   '原发性高血压',
   '十二指肠胃反流及胆汁反流性胃炎',
   '卵巢早衰',
   '盆腔淤血综合征',
   '酒中毒性周围神经病',
   '男性更年期综合症',
   '汞中毒',
   '纤维肌痛综合征',
   '肝炎后综合征',
   '复发性阿弗他性口腔炎',
   '心脏神经官能症',
   '失眠症',
   '焦虑症',
   '老年人神经症',
   '工业中毒所致的精神障碍',
   '心血管疾病伴发的精神障碍',
   '慢性硬脑膜下血肿']}]

In [7]:
zzs = list({i['症状名称'] for i in zzlist})
len(zzs)

3488

In [29]:
diseases1 = set()
for i in zzlist:
    diseases1 = diseases1.union(i['相关疾病'])
diseases1 = list(diseases1)
len(diseases1)

6482

### 去掉极严重的病  
罕见程度 == 0  or 不在疾病库中

In [26]:
distable = pmw.get_collection('jiankang39','diseases')

In [27]:
r = set()

In [31]:
for i in diseases1:
    d = pmw.find_one(distable,{'疾病名称':i},fieldlist=['罕见程度'])
    if d:
        if d['罕见程度'] == 1 or d['罕见程度'] == 2:
            r.add(i)
    else:
        print(i)
        r.add(i)
len(r)

性厌恶
腺瘤
胸腺小细胞癌
死精症
男性假两性畸形
痞气
光气中毒
小儿触电与雷击
精瘀证
消化系统溃疡病
无精症
产后风
副球孢子菌病
女性生殖道多部位原发癌
内伤发热
乙状结肠间疝
苯海拉明中毒
小儿单纯性血尿
骨纤维肉瘤
蓐劳
增生性骨关节病
小儿其他沙门菌感染
特发性高嗜酸性细胞综合征
骨软骨瘤
肤胀
女性的青春期延迟
精索恶性肿瘤
慢性肠炎
淋病合并症精囊炎
带下病
男性生殖器外伤
真两性畸形
菌血症
真菌性食物中毒
鱼胆中毒
空调病
代谢性酸中毒
组织胞浆菌病
节育手术并发症
斑痧
回归热
慢性附件炎
疝气
老年人骨质疏松
全植物神经功能不全
水臌
中满
消中
脐疝
婴儿脐疝
骨母细胞瘤
对乙酰氨基酚中毒
软骨发育异常
不射精症
夏季低热
进行性骨干发育不全
骨雅司病
男性生殖腺机能减退症
骨囊肿
妇女多毛症
婴幼儿脐疝
纹状骨瘤
四氯化碳中毒
黄曲霉素中毒
痢疾
痃癖
多发性骨骺发育异常
转移性胸膜肿瘤
生长期脱发
骨化性肌炎
女性慢性淋病
软组织肿瘤
白线疝
小棘毛壅病
早产
淋球菌性腹膜炎
宿醉
性早熟
下消
造口旁疝
乳糜胸
腰骶部脂肪疝
内生性软骨瘤
黏多糖贮积症Ⅶ型
胁痛
黏多糖贮积症Ⅳ型
假斑秃
妊娠合并生殖器疱疹
男科外伤
产后虚羸
朗格汉斯组织细胞增多症
生长痛
急性脓胸
血热崩漏
胸腺瘤
反分化软骨肉瘤
胸膜炎
念珠状发
骨炎
气胀
拉休尤-格拉哈姆-利特尔综合征
子满
小儿两性畸形
经行头痛
妇女雄激素源性秃发
腹部创伤
尿精
坐月子
Q热
隆突性皮肤纤维肉瘤
妊娠合并支原体感染
腹膜后脂肪瘤
腹卒中综合征
脐部脐肠系膜管息肉
生乳头汗管囊腺瘤
经行发热
风邪致病
维生素中毒
C5功能不全综合征
继发性痛经
精囊囊肿
子宫复旧不全
小儿疳痢
下背部痛
急性腰扭伤
少腹如扇
脓胸
性交疼痛
痛性肥胖病
锌缺乏病
不孕症
男性不育
妊娠合并生殖道沙眼衣原体感染
腹腔室隔综合征
精囊恶性肿瘤
软骨肉瘤
点状骨骺发育异常
男性生殖器畸形
颈部软组织损伤
小儿46-XY单纯性腺发育不全综合征
盆腔炎性肿块
食物不耐受
淋病
寄生虫病
房事两感症
真菌中毒症
新生儿B链球菌感染
小儿生殖细胞瘤
阴盛阳衰症
虚劳
腰疝
月经失调性不孕症
小儿泄泻
腹绞痛综合征
毛囊性黏蛋白病
五氯酚钠中毒
骨嗜酸性肉芽肿
进行性骨干发育不良
膀胱上内疝
腰肌劳损

980

In [34]:
diseases = [i for i in diseases1 if i not in r]
len(diseases)

5502

In [9]:
from utils.pandas_wrapper import pandas_wrapper as pdw 

In [10]:
zzseries = pdw.build_series(zzs)
zzseries[:5]

0          指甲翘起
1       扁桃体充血肥大
2      拇指不明原因疼痛
3          咽喉疼痛
4    头或颈部发胀及跳动感
dtype: object

In [151]:
pkw.dump2file(zzseries,'/home/ian/code/data/daozhen/zzseries.pkl')

In [33]:
disseries = pdw.build_series(diseases)
disseries[:5]

0       肝动脉闭塞
1    急性化脓性腮腺炎
2        赤脉传睛
3       肛门瘙痒症
4        支气管炎
dtype: object

In [150]:
pkw.dump2file(disseries,'/home/ian/code/data/daozhen/disseries.pkl')

In [12]:
zzseries_inv = pdw.build_series(zzseries.index,index=zzseries.values)
zzseries_inv[:5]

指甲翘起          0
扁桃体充血肥大       1
拇指不明原因疼痛      2
咽喉疼痛          3
头或颈部发胀及跳动感    4
dtype: int64

In [135]:
pkw.dump2file(zzseries_inv,'/home/ian/code/data/daozhen/zzseries_inv.pkl')

In [35]:
disseries_inv = pdw.build_series(disseries.index,index=disseries.values)
disseries_inv[:5]

肝动脉闭塞       0
急性化脓性腮腺炎    1
赤脉传睛        2
肛门瘙痒症       3
支气管炎        4
dtype: int64

### 构建疾病症状矩阵

In [40]:
from utils.numpy_wrapper import numpy_wrapper as npw

zzdismatr = npw.build_zeros_array((len(zzseries),len(disseries)))

for i in zzlist:
    x = zzseries_inv[i['症状名称']]
    for ii in i['相关疾病']:
        if ii in disseries_inv:
            zzdismatr[x,disseries_inv[ii]] = 1

diszzmatr = zzdismatr.T

In [134]:
pkw.dump2file(diszzmatr,'/home/ian/code/data/daozhen/diszzmatr.pkl')

In [148]:
diszzmatr.nonzero()

(array([   0,    0,    1, ..., 5500, 5501, 5501]),
 array([ 517, 3291,   64, ..., 3370,  184, 2225]))

In [20]:
zz1 = ['指甲翘起']
temp = npw.build_zeros_array((1,len(zzseries)))[0]
for i in zz1:
    temp[zzseries_inv[i]] = 1

In [21]:
from math import sqrt
def cal_cos_similarity(a1,a2):
    a = a1.dot(a2)
    b = a1.dot(a1)
    c = a2.dot(a2)
    if b == 0 or c==0:
        return 0
    return a/sqrt( b*c )

In [39]:
diszzmatr[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [44]:
diszzmatr.shape

(5502, 3488)

In [23]:
temp.shape

(3488,)

In [105]:
#输出可能的疾病和概率
def zz2disease(zzs,gender='男',age='4',other='普通'):
    genderdict = {'男':0,'女':1}
    otherdict = {'普通':[0],'孕期':[1],'产褥期':[2],'孕妇及产褥期':[1,2]}
    gender = genderdict[gender]
    other = otherdict[other]
    temp = npw.build_zeros_array((1,len(zzseries)))[0]
    for i in zzs:
        temp[zzseries_inv[i]] = 1
    l1 = list()
    for i in range(len(disseries)):
        l1.append(cal_cos_similarity(temp,diszzmatr[i]))
    s1 = pdw.build_series(l1)
    s2 = s1.sort_values(ascending=False)
    diss = list(disseries[s2.iloc[s2.nonzero()].index].values)
    dissprob = list(s2.iloc[s2.nonzero()].values)
    for i in range(len(diss)):
        d = pmw.find_one(distable,{'疾病名称':diss[i],'限定性别':{'$in':[gender]},'限定年龄':{'$in':[age]},'备注':{'$in':other}},fieldlist=['罕见程度','多发年龄','多发性别'])
        if d:
            if d['罕见程度'] == 1:
                dissprob[i] *= 0.1
            if age not in d['多发年龄']:
                dissprob[i] *= 0.1
            if gender not in d['多发性别']:
                dissprob[i] *= 0.1
        else:
            dissprob[i] = 0
    df = pdw.build_df_from_dict({'疾病':diss,'概率':dissprob})    
    return df.sort_values('概率',ascending=False)

In [145]:
zz2disease(['鼻塞', '咳嗽', '呼吸困难', '流黄鼻涕', '头痛'])

,疾病,概率
0,虚寒咳嗽,0.547723
1,风热犯肺,0.474342
2,风温,0.447214
3,鼻石,0.447214
4,鼻中隔脓肿,0.447214
5,风热头痛,0.365148
12,头风病,0.316228
19,鼻咽肉瘤,0.316228
18,鼻咽炎,0.316228
16,嗜酸细胞增多性非变态反应性鼻炎,0.316228


In [106]:
zz1 = ['牙龈出血']
zz2disease(zz1)

,疾病,概率
0,血小板无力症,1.000000
3,血小板型假性血管性血友病,0.707107
5,δ-贮存池病,0.707107
6,药物免疫性血小板减少性紫癜,0.707107
1,α-贮存池病,0.707107
7,真性红细胞增多症,0.577350
8,黑热病,0.577350
12,血管性血友病,0.500000
19,轻链病与轻链沉积病,0.500000
13,牙宣,0.500000


In [107]:
zz1 = ['恶心','头痛','乏力','健忘']
zz2disease(zz1)

,疾病,概率
0,蛋白胨性荨麻疹,0.707107
2,慢性鼻咽炎,0.530330
1,鼻咽炎,0.530330
3,胃内异物,0.500000
5,胰腺肿瘤,0.447214
7,急性无黄疸型肝炎,0.408248
8,阑尾炎,0.353553
9,头风病,0.353553
16,中性粒细胞减少症,0.353553
12,功能性呕吐,0.353553


In [108]:
zz1 = ['小腿酸痛','小腿发胀','脚后跟疼']
zz2disease(zz1)

,疾病,概率
0,情感性交叉擦腿症,0.333333
1,原发性下肢静脉瓣膜关闭不全,0.288675
2,血栓闭塞性脉管炎,0.258199
3,下肢静脉血栓,0.258199
5,腰腿痛,0.204124
7,慢性单纯外伤性腰腿痛,0.154303
4,老年静脉血栓症,0.000000
6,急性单纯外伤性腰腿痛,0.000000


In [124]:
distable = pmw.get_collection('jiankang39','diseases')

In [64]:
from utils.pickle_wrapper import pickle_wrapper as pkw

In [68]:
ksdicts = pkw.loadfromfile('/home/ian/code/data/daozhen/ksdict.pkl')
ksdicts

{'胃肠外科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '外伤科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '性病科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '男科': ([0], ['4', '5', '6', '7'], [0, 1, 2]),
 '手外科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '传染科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '肾内科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '皮肤科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '呼吸内科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '内科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '消化内科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '内分泌科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '肛肠外科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '儿科': ([0, 1], ['1', '2', '3'], [0, 1, 2]),
 '妇产科': ([1], ['4', '5', '6'], [0, 1, 2]),
 '血液科': ([0, 1], ['1', '2', '3', '4', '5', '6', '7'], [0, 1, 2]),
 '乳

In [124]:
def zz2ks(zzs,gender='男',age='4',other='普通'):
    df = zz2disease(zzs,gender,age,other)
    a = list(df['疾病'])
    b = list(df['概率'])
    genderdict = {'男':0,'女':1}
    otherdict = {'普通':[0],'孕期':[1],'产褥期':[2],'孕妇及产褥期':[1,2]}
    gender = genderdict[gender]
    other = otherdict[other]
    ksdict = dict()
    for i in range(len(a)):
        ks = distable.find_one({'疾病名称':a[i]})
        if ks and '挂号的科室' in ks:
            ks = ks['挂号的科室'].split()
            for ii in ks:
                if ii in ksdicts and age in ksdicts[ii][1] and gender in ksdicts[ii][0] and other[0] in ksdicts[ii][2]:
                    print(a[i],ks,b[i])
                    if ii in ksdict:
                        ksdict[ii] += b[i]
                    else:
                        ksdict[ii] = b[i]
    if '血液科' in ksdict:
        ksdict['血液科'] = ksdict['血液科'] / 6        
    s = pdw.build_series(ksdict).sort_values(ascending=False)
    s = s/((s.iloc[0]+1))
    return s

In [96]:
list(disseries[s2.iloc[s2.nonzero()].index].values)

['颅内动脉瘤', '小儿甲状旁腺功能减退症']

In [125]:
zz2ks(['小腿酸痛','小腿发胀','脚后跟疼'])

原发性下肢静脉瓣膜关闭不全 ['血管外科'] 0.2886751345948129
血栓闭塞性脉管炎 ['血管外科'] 0.2581988897471611
下肢静脉血栓 ['血管外科'] 0.2581988897471611
腰腿痛 ['老年科', '普外科'] 0.20412414523193154
慢性单纯外伤性腰腿痛 ['骨科'] 0.1543033499620919
老年静脉血栓症 ['血管外科', '老年科'] 0.0
急性单纯外伤性腰腿痛 ['外伤科', '骨科'] 0.0
急性单纯外伤性腰腿痛 ['外伤科', '骨科'] 0.0


血管外科    0.446006
普外科     0.113084
骨科      0.085483
外伤科     0.000000
dtype: float64

In [126]:
zz2ks(['恶心','头痛','乏力','健忘'])

蛋白胨性荨麻疹 ['皮肤科'] 0.7071067811865475
慢性鼻咽炎 ['耳鼻喉科'] 0.5303300858899106
鼻咽炎 ['耳鼻喉科'] 0.5303300858899106
胃内异物 ['消化内科'] 0.5
胰腺肿瘤 ['肿瘤科', '肝胆外科'] 0.4472135954999579
胰腺肿瘤 ['肿瘤科', '肝胆外科'] 0.4472135954999579
急性无黄疸型肝炎 ['消化内科', '传染科'] 0.4082482904638631
急性无黄疸型肝炎 ['消化内科', '传染科'] 0.4082482904638631
阑尾炎 ['消化内科', '胃肠外科'] 0.35355339059327373
阑尾炎 ['消化内科', '胃肠外科'] 0.35355339059327373
头风病 ['中医科'] 0.35355339059327373
中性粒细胞减少症 ['血液科'] 0.35355339059327373
功能性呕吐 ['消化内科', '内科'] 0.35355339059327373
功能性呕吐 ['消化内科', '内科'] 0.35355339059327373
上颌窦炎 ['耳鼻喉科'] 0.35355339059327373
Castleman病 ['血液科', '内分泌科'] 0.35355339059327373
Castleman病 ['血液科', '内分泌科'] 0.35355339059327373
原发性高血压 ['心血管内科'] 0.2886751345948129
原发性骨髓纤维化 ['血液科'] 0.2886751345948129
特发性室性心动过速 ['心血管内科'] 0.2886751345948129
葡萄球菌感染 ['内科', '传染科'] 0.2886751345948129
葡萄球菌感染 ['内科', '传染科'] 0.2886751345948129
肝阳上亢 ['中医科'] 0.2886751345948129
房事头疼症 ['中医科'] 0.2886751345948129
溃疡病出血 ['消化内科', '胃肠外科'] 0.2886751345948129
溃疡病出血 ['消化内科', '胃肠外科'] 0.2886751345948129
慢性荨麻疹 ['皮肤科'

中医科      0.867540
消化内科     0.693096
呼吸内科     0.245957
传染科      0.244842
耳鼻喉科     0.220442
心理咨询     0.205969
皮肤科      0.188432
肿瘤科      0.165777
内科       0.163677
内分泌科     0.161470
眼科       0.160583
心血管内科    0.158919
肝胆外科     0.158028
性病科      0.104731
胃肠外科     0.085070
男科       0.078067
营养科      0.060153
肝炎       0.055192
血液科      0.048670
普外科      0.046832
肾内科      0.036704
心胸外科     0.036554
血管外科     0.035618
神经内科     0.035490
肝病       0.033115
精神病科     0.027038
体检保健科    0.026901
口腔科      0.023802
泌尿外科     0.023416
肛肠外科     0.023416
脑外科      0.020391
风湿科      0.016558
急诊科      0.010769
dtype: float64

In [127]:
zz2ks(['牙龈出血'])

血小板无力症 ['血液科'] 1.0
血小板型假性血管性血友病 ['血液科'] 0.7071067811865475
δ-贮存池病 ['血液科'] 0.7071067811865475
药物免疫性血小板减少性紫癜 ['血液科'] 0.7071067811865475
α-贮存池病 ['血液科'] 0.7071067811865475
真性红细胞增多症 ['血液科'] 0.5773502691896258
黑热病 ['传染科'] 0.5773502691896258
血管性血友病 ['血液科'] 0.5
轻链病与轻链沉积病 ['血液科', '肾内科'] 0.5
轻链病与轻链沉积病 ['血液科', '肾内科'] 0.5
牙宣 ['口腔科', '中医科'] 0.5
牙宣 ['口腔科', '中医科'] 0.5
急性失血所致贫血 ['血液科', '内科'] 0.4472135954999579
急性失血所致贫血 ['血液科', '内科'] 0.4472135954999579
维生素K缺乏病 ['血液科', '内科'] 0.4472135954999579
维生素K缺乏病 ['血液科', '内科'] 0.4472135954999579
肥大性龈炎 ['口腔科'] 0.4472135954999579
暴发性肝炎样综合征 ['肝炎', '传染科'] 0.4082482904638631
暴发性肝炎样综合征 ['肝炎', '传染科'] 0.4082482904638631
急性白血病 ['血液科'] 0.4082482904638631
肝纤维化 ['肝病', '消化内科'] 0.3779644730092272
肝纤维化 ['肝病', '消化内科'] 0.3779644730092272
特发性血小板减少性紫癜 ['血液科', '内科'] 0.35355339059327373
特发性血小板减少性紫癜 ['血液科', '内科'] 0.35355339059327373
坏血病 ['儿科', '营养科'] 0.31622776601683794
牙周炎 ['口腔科'] 0.25
牙周病 ['口腔科'] 0.17149858514250882
严重肝病引起获得性凝血因子异常 ['血液科'] 0.07071067811865475
登革出血热 ['传染科', '急诊科'] 0.07

口腔科     0.577830
内科      0.526860
血液科     0.509995
传染科     0.445942
中医科     0.211085
肾内科     0.211085
肝炎      0.172350
消化内科    0.159565
肝病      0.159565
营养科     0.133502
急诊科     0.045808
肿瘤科     0.024374
肝胆外科    0.000000
外伤科     0.000000
dtype: float64

In [128]:
zz2ks(['男子性功能障碍'])

血色病性心肌病 ['心血管内科', '心胸外科'] 0.7071067811865475
血色病性心肌病 ['心血管内科', '心胸外科'] 0.7071067811865475
早泄 ['男科', '生殖健康'] 0.5773502691896258
早泄 ['男科', '生殖健康'] 0.5773502691896258
继发性早泄 ['男科', '泌尿外科'] 0.5773502691896258
继发性早泄 ['男科', '泌尿外科'] 0.5773502691896258
原发性早泄 ['男科', '泌尿外科'] 0.5773502691896258
原发性早泄 ['男科', '泌尿外科'] 0.5773502691896258
阳痿 ['男科', '生殖健康'] 0.5773502691896258
阳痿 ['男科', '生殖健康'] 0.5773502691896258
阴茎异常勃起 ['男科', '泌尿外科'] 0.5773502691896258
阴茎异常勃起 ['男科', '泌尿外科'] 0.5773502691896258
肝硬化男性性功能减弱综合征 ['消化内科', '男科'] 0.5
肝硬化男性性功能减弱综合征 ['消化内科', '男科'] 0.5
第Ⅱ、Ⅲ型脊髓血管畸形 ['心血管内科', '血管外科'] 0.5
第Ⅱ、Ⅲ型脊髓血管畸形 ['心血管内科', '血管外科'] 0.5
肾精亏虚 ['中医科'] 0.4472135954999579
小阴茎 ['男科'] 0.4082482904638631
精囊炎 ['男科', '泌尿外科'] 0.4082482904638631
精囊炎 ['男科', '泌尿外科'] 0.4082482904638631
围术期伴发的精神障碍 ['心理咨询', '精神病科'] 0.4082482904638631
围术期伴发的精神障碍 ['心理咨询', '精神病科'] 0.4082482904638631
维生素E缺乏病 ['内科', '儿科'] 0.4082482904638631
克兰费尔特综合征 ['内分泌科', '中医科'] 0.4082482904638631
克兰费尔特综合征 ['内分泌科', '中医科'] 0.4082482904638631
慢性细菌性前列腺炎 ['泌尿外科', '男科'] 0

男科       0.832568
泌尿外科     0.487163
心血管内科    0.202109
生殖健康     0.193334
中医科      0.181644
内分泌科     0.134579
心胸外科     0.118393
神经内科     0.118393
消化内科     0.095555
血管外科     0.095555
心理咨询     0.068354
内科       0.068354
精神病科     0.068354
针灸科      0.059196
呼吸内科     0.052947
耳鼻喉科     0.052947
肾内科      0.038412
血液科      0.009866
肿瘤科      0.000000
脑外科      0.000000
性病科      0.000000
dtype: float64

In [155]:
for i,n in zz2ks(['头胀','单侧头痛'])[:3].items():
    print(i,n)
    break

瘀血头痛 ['中医科'] 0.5345224838248488
小儿高血压 ['心血管内科', '儿科'] 0.35355339059327373
肝炎后综合征 ['肝炎'] 0.35355339059327373
夏季感冒 ['中医科', '呼吸内科'] 0.23570226039551587
夏季感冒 ['中医科', '呼吸内科'] 0.23570226039551587
腹型偏头痛综合征 ['消化内科', '神经内科'] 0.21320071635561041
腹型偏头痛综合征 ['消化内科', '神经内科'] 0.21320071635561041
原发性高血压 ['心血管内科'] 0.20412414523193154
鼻咽癌 ['肿瘤科', '耳鼻喉科'] 0.1889822365046136
鼻咽癌 ['肿瘤科', '耳鼻喉科'] 0.1889822365046136
痰浊头痛 ['中医科', '呼吸内科'] 0.1543033499620919
痰浊头痛 ['中医科', '呼吸内科'] 0.1543033499620919
颅骨结核 ['脑外科', '传染科'] 0.035355339059327376
颅骨结核 ['脑外科', '传染科'] 0.035355339059327376
偏头痛 ['神经内科'] 0.0316227766016838
高颅压性脑积水 ['脑外科', '神经内科'] 0.025
高颅压性脑积水 ['脑外科', '神经内科'] 0.025
颅内高压综合征 ['神经内科', '脑外科'] 0.0040824829046386315
颅内高压综合征 ['神经内科', '脑外科'] 0.0040824829046386315
颅内动脉瘤 ['肿瘤科', '脑外科'] 0.00028867513459481295
颅内动脉瘤 ['肿瘤科', '脑外科'] 0.00028867513459481295
小儿颅内肿瘤 ['神经内科', '脑外科'] 0.0
小儿颅内肿瘤 ['神经内科', '脑外科'] 0.0
新生儿颅内出血 ['儿科', '脑外科'] 0.0
老年人颅内压增高 ['神经内科', '脑外科'] 0.0
老年人颅内压增高 ['神经内科', '脑外科'] 0.0
小儿后颅窝型脑积水综合征 ['脑外科', '儿科'] 0.0


In [149]:
zz2ks(['牙龈肿胀', '牙的咀嚼磨损', '发烧', '牙疼', '牙龈溢脓', '齿龈炎'])

牙髓炎 ['口腔科'] 0.7071067811865476
边缘性龈炎 ['口腔科'] 0.43301270189221935
急性颌骨骨髓炎 ['口腔科'] 0.43301270189221935
牙隐裂 ['口腔科'] 0.4082482904638631
EB病毒感染 ['传染科', '皮肤科'] 0.4082482904638631
EB病毒感染 ['传染科', '皮肤科'] 0.4082482904638631
中间型链球菌感染 ['皮肤科', '传染科'] 0.4082482904638631
中间型链球菌感染 ['皮肤科', '传染科'] 0.4082482904638631
药物性牙龈增生 ['口腔科'] 0.4082482904638631
急性脓肿 ['外科', '内科'] 0.4082482904638631
急性脓肿 ['外科', '内科'] 0.4082482904638631
颌骨骨髓炎 ['口腔科', '骨科'] 0.3692744729379982
颌骨骨髓炎 ['口腔科', '骨科'] 0.3692744729379982
肥大性龈炎 ['口腔科'] 0.3651483716701107
牙痈 ['中医科', '口腔科'] 0.3086066999241838
牙痈 ['中医科', '口腔科'] 0.3086066999241838
鼻窦气压性创伤 ['耳鼻喉科', '外伤科'] 0.2886751345948129
鼻窦气压性创伤 ['耳鼻喉科', '外伤科'] 0.2886751345948129
青少年牙周炎 ['口腔科'] 0.2886751345948129
智齿冠周炎 ['口腔科'] 0.23570226039551587
胃穿孔 ['胃肠外科'] 0.23570226039551587
风疹 ['传染科', '皮肤科'] 0.23570226039551587
风疹 ['传染科', '皮肤科'] 0.23570226039551587
上颌窦恶性肿瘤 ['肿瘤科', '耳鼻喉科'] 0.23570226039551587
上颌窦恶性肿瘤 ['肿瘤科', '耳鼻喉科'] 0.23570226039551587
疫黄 ['中医科'] 0.23570226039551587
遗传性乳光牙 ['口腔科'] 0.2182178

口腔科      0.842776
中医科      0.386104
呼吸内科     0.248466
传染科      0.239390
皮肤科      0.235359
眼科       0.192929
耳鼻喉科     0.114538
消化内科     0.089827
外科       0.064186
内科       0.064186
骨科       0.058059
肿瘤科      0.051916
性病科      0.050101
肝胆外科     0.048897
外伤科      0.045387
胃肠外科     0.037058
风湿科      0.026204
肝炎       0.022693
心血管内科    0.021395
营养科      0.020298
肾内科      0.016573
急诊科      0.006190
心胸外科     0.004539
内分泌科     0.002269
心理咨询     0.001338
血液科      0.000000
dtype: float64

In [131]:
zz2ks(['后脑勺阵痛','头部撞伤','头晕'])

高温引起的溶血性贫血 ['血液科', '烧伤科'] 0.3333333333333333
高温引起的溶血性贫血 ['血液科', '烧伤科'] 0.3333333333333333
房事头疼症 ['中医科'] 0.3333333333333333
厥阴头痛 ['中医科'] 0.2886751345948129
镰状细胞贫血眼部病变 ['眼科', '血液科'] 0.2886751345948129
镰状细胞贫血眼部病变 ['眼科', '血液科'] 0.2886751345948129
电脑视觉综合症 ['眼科', '营养科'] 0.2886751345948129
电脑视觉综合症 ['眼科', '营养科'] 0.2886751345948129
药物相关性免疫性溶血性贫血 ['血液科'] 0.2581988897471611
郁冒 ['中医科'] 0.2581988897471611
阳虚发热 ['中医科'] 0.2581988897471611
冷抗体型自身免疫性溶血性贫血 ['血液科'] 0.23570226039551587
砷化氢引起的溶血性贫血 ['血液科', '急诊科'] 0.23570226039551587
砷化氢引起的溶血性贫血 ['血液科', '急诊科'] 0.23570226039551587
热咳 ['中医科'] 0.23570226039551587
痰积 ['中医科', '呼吸内科'] 0.2182178902359924
痰积 ['中医科', '呼吸内科'] 0.2182178902359924
体质性低血压 ['中医科'] 0.20412414523193154
脾水 ['中医科'] 0.20412414523193154
慢性鼻咽炎 ['耳鼻喉科'] 0.20412414523193154
血虚眩晕 ['中医科'] 0.20412414523193154
铜中毒引起的溶血性贫血 ['血液科', '急诊科'] 0.20412414523193154
铜中毒引起的溶血性贫血 ['血液科', '急诊科'] 0.20412414523193154
遗传性铁粒幼细胞性贫血 ['血液科'] 0.20412414523193154
鼻咽炎 ['耳鼻喉科'] 0.20412414523193154
肝火犯肺 ['中医科', '呼吸内科'] 0.2041

中医科      0.789243
眼科       0.153098
呼吸内科     0.121532
急诊科      0.101651
耳鼻喉科     0.086041
烧伤科      0.070252
血液科      0.069929
营养科      0.060840
心理咨询     0.043021
性病科      0.043021
消化内科     0.039190
心血管内科    0.038479
皮肤科      0.031418
脑外科      0.022995
神经内科     0.022995
体检保健科    0.019003
dtype: float64

In [144]:
zz2ks(['恶心', '失眠', '焦虑', '头痛', '抑郁', '头沉', '疲劳'])

空巢综合症 ['心理咨询'] 0.3779644730092272
疑病症 ['精神病科', '心理咨询'] 0.3779644730092272
疑病症 ['精神病科', '心理咨询'] 0.3779644730092272
经行失眠 ['中医科'] 0.3779644730092272
原发免疫性疾病相关的淋巴增殖性疾病 ['血液科'] 0.3779644730092272
胃内异物 ['消化内科'] 0.3779644730092272
电脑狂躁症 ['心理咨询'] 0.3779644730092272
骨髓痨性贫血 ['血液科', '内科'] 0.3779644730092272
骨髓痨性贫血 ['血液科', '内科'] 0.3779644730092272
周末焦虑症 ['心理咨询'] 0.3779644730092272
外隐斜 ['眼科', '外科'] 0.3779644730092272
外隐斜 ['眼科', '外科'] 0.3779644730092272
X综合征 ['心血管内科'] 0.3779644730092272
密码综合症 ['心理咨询'] 0.3779644730092272
考后综合症 ['心理咨询'] 0.3380617018914066
血小板减少症 ['血液科'] 0.3086066999241838
网瘾 ['心理咨询'] 0.2857142857142857
克雅氏病 ['神经内科'] 0.2857142857142857
淋病后综合征 ['性病科', '心理咨询'] 0.2672612419124244
淋病后综合征 ['性病科', '心理咨询'] 0.2672612419124244
肺炎衣原体感染 ['呼吸内科', '传染科'] 0.2672612419124244
肺炎衣原体感染 ['呼吸内科', '传染科'] 0.2672612419124244
洁癖 ['精神心理科', '心理咨询'] 0.2672612419124244
洁癖 ['精神心理科', '心理咨询'] 0.2672612419124244
特发性嗜酸性粒细胞增多综合征 ['血液科'] 0.2672612419124244
蛋白胨性荨麻疹 ['皮肤科'] 0.2672612419124244
乡愁综合症 ['心理咨询'] 0.267261241912

中医科       0.864064
心理咨询      0.762199
消化内科      0.580252
传染科       0.351371
神经内科      0.324447
心血管内科     0.322222
精神病科      0.310210
内科        0.247293
呼吸内科      0.217996
眼科        0.193767
耳鼻喉科      0.146468
皮肤科       0.133125
内分泌科      0.119961
风湿科       0.106067
肿瘤科       0.096925
肝胆外科      0.092028
体检保健科     0.081272
精神心理科     0.080880
性病科       0.076434
急诊科       0.071731
血液科       0.068653
心胸外科      0.058021
营养科       0.055353
外科        0.051379
胃肠外科      0.049542
中西医结合科    0.048667
肝炎        0.042816
肾内科       0.038252
肝病        0.025690
脑外科       0.019419
口腔科       0.018465
普外科       0.018165
骨科        0.016248
皮肤性病      0.013412
血管外科      0.003884
泌尿外科      0.002298
dtype: float64

In [137]:
dict(zz2ks(['后脑勺阵痛','头部撞伤','头晕'])[:3])

高温引起的溶血性贫血 ['血液科', '烧伤科'] 0.3333333333333333
高温引起的溶血性贫血 ['血液科', '烧伤科'] 0.3333333333333333
房事头疼症 ['中医科'] 0.3333333333333333
厥阴头痛 ['中医科'] 0.2886751345948129
镰状细胞贫血眼部病变 ['眼科', '血液科'] 0.2886751345948129
镰状细胞贫血眼部病变 ['眼科', '血液科'] 0.2886751345948129
电脑视觉综合症 ['眼科', '营养科'] 0.2886751345948129
电脑视觉综合症 ['眼科', '营养科'] 0.2886751345948129
药物相关性免疫性溶血性贫血 ['血液科'] 0.2581988897471611
郁冒 ['中医科'] 0.2581988897471611
阳虚发热 ['中医科'] 0.2581988897471611
冷抗体型自身免疫性溶血性贫血 ['血液科'] 0.23570226039551587
砷化氢引起的溶血性贫血 ['血液科', '急诊科'] 0.23570226039551587
砷化氢引起的溶血性贫血 ['血液科', '急诊科'] 0.23570226039551587
热咳 ['中医科'] 0.23570226039551587
痰积 ['中医科', '呼吸内科'] 0.2182178902359924
痰积 ['中医科', '呼吸内科'] 0.2182178902359924
体质性低血压 ['中医科'] 0.20412414523193154
脾水 ['中医科'] 0.20412414523193154
慢性鼻咽炎 ['耳鼻喉科'] 0.20412414523193154
血虚眩晕 ['中医科'] 0.20412414523193154
铜中毒引起的溶血性贫血 ['血液科', '急诊科'] 0.20412414523193154
铜中毒引起的溶血性贫血 ['血液科', '急诊科'] 0.20412414523193154
遗传性铁粒幼细胞性贫血 ['血液科'] 0.20412414523193154
鼻咽炎 ['耳鼻喉科'] 0.20412414523193154
肝火犯肺 ['中医科', '呼吸内科'] 0.2041

{'中医科': 0.789242906398993,
 '眼科': 0.15309847725964038,
 '呼吸内科': 0.1215321054716626}

In [138]:
import pandas as pd

In [139]:
s = pd.Series([])

In [142]:
if not s.empty:
    print(s)